In [ ]:
import cv2
import time
import json
import re
import requests
from collections import defaultdict
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import warnings
import os
from tiki.mini import TikiMini
from ultralytics import YOLO

# 분리된 모듈 import
from utils.config import (
    MARKER_TO_POINT, OBJECT_NAMES, BASE_SPEED, MAX_STEERING, FRAME_CENTER_X,
    BIRD_WIDTH, BIRD_HEIGHT, SRC_POINTS, DST_POINTS, JSON_PATH, SERVER_URL
)
from utils.pid_controller import PIDController
from utils.server_communication import send_to_server, send_dashboard_image, startSend
from utils.lane_detection import detect_lane_center, make_panel
from utils.pothole_detection import check_and_handle_pothole
from utils.image_utils import frame_to_bytes_drive, frame_to_bytes_yolo

warnings.filterwarnings('ignore', category=UserWarning)
os.environ['GST_DEBUG'] = '0'

##############################################
# 1. YOLOv8 학습 모델 로드 (가장 먼저 실행)
##############################################
model = YOLO("./best4.pt")  # 🔥 학습한 best.pt 경로 확인
print("[YOLO] 모델 로드 완료")

##############################################
# 1-1. A3R8.json 파일 대기 (파일이 생성될 때까지 무한 대기)
##############################################
print(f"[WAIT] JSON 파일 대기 중: {JSON_PATH}")
while not os.path.exists(JSON_PATH):
    print(f"[WAIT] JSON 파일 대기 중... (0.5초마다 확인)")
    time.sleep(0.5)


# 파일이 생성되었지만 아직 쓰기 중일 수 있으므로, 읽기 성공할 때까지 재시도
max_retries = 10
retry_count = 0
json_loaded = False

while not json_loaded and retry_count < max_retries:
    try:
        with open(JSON_PATH, 'r', encoding='utf-8') as f:
            test_data = json.load(f)
        json_loaded = True
        print(f"[SUCCESS] JSON 파일 로드 완료: {JSON_PATH}")
        print(f"  mission_code: {test_data.get('mission_code', 'N/A')}")
    except (json.JSONDecodeError, IOError) as e:
        retry_count += 1
        if retry_count < max_retries:
            print(f"[RETRY] JSON 파일 읽기 재시도 ({retry_count}/{max_retries}): {e}")
            time.sleep(0.2)
        else:
            print(f"[WARNING] JSON 파일 읽기 최종 실패: {e}, 계속 진행합니다.")

##############################################
# 1-2. 시작 통신 전송 (startdashboard 로직)
startSend()
##############################################

##############################################
# 2. ArUco 설정
##############################################
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
aruco_detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

# 설정에서 마커 매핑 가져오기
marker_to_point = MARKER_TO_POINT
object_names = OBJECT_NAMES

visited_points = set()          # internal set to avoid duplicate handling
transmitted_points = []         # ordered list to send in payload's `points` array

# 각 point별로 누적된 detection 정보 저장 {point_name: {class_name: count}}
point_detections = defaultdict(lambda: defaultdict(int))
current_point = None  # 현재 위치한 point (마지막으로 지나간 point)
visited_sectors = set()  # 이미 이미지를 전송한 sector 추적 (중복 전송 방지)

##############################################
# 3. 로봇 초기화
##############################################
tiki = TikiMini()
tiki.set_motor_mode(tiki.MOTOR_MODE_PWM)
# 안전: 초기화 직후 모터 정지
try:
    tiki.stop()
except Exception:
    pass
print("로봇 초기화 완료 (모터 정지)")

# PID 제어
pid = PIDController()

##############################################
# 4. 단일 카메라 파이프라인 (주행 + YOLO 공용)
##############################################
pipeline = (
    "nvarguscamerasrc ! "
    "video/x-raw(memory:NVMM), width=640, height=480, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! "
    "video/x-raw, format=BGR ! appsink drop=true max-buffers=1"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
if not cap.isOpened():
    raise RuntimeError("카메라를 열 수 없습니다.")

# 최신 프레임만 사용하도록 버퍼 최소화
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

##############################################
# 5. Jupyter용 두 화면 위젯 (주행 / 객체인식)
##############################################
drive_widget = widgets.Image(
    format='jpeg',
    layout=widgets.Layout(width='640px', height='480px')
)
yolo_widget = widgets.Image(
    format='jpeg',
    layout=widgets.Layout(width='640px', height='480px')
)

display(widgets.HBox([drive_widget, yolo_widget]))

##############################################
# 6. Bird's-Eye View 설정 (주행용)
##############################################
M = cv2.getPerspectiveTransform(SRC_POINTS, DST_POINTS)

##############################################
# 7. 메인 루프 (단일 카메라 → 주행 + YOLO 두 화면 출력)
##############################################
print("[START] 주행 + YOLO + ArUco 시스템 실행")

frame_idx = 0
annotated = None  # 마지막 YOLO 결과 프레임 보관

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            time.sleep(8)
            continue

        frame_idx += 1

        # 공통 원본 프레임 (카메라 기준 회전 보정)
        frame = cv2.flip(frame, -1)

        # 주행용 처리를 위해 320x240으로 다운스케일한 프레임 사용
        frame_small = cv2.resize(frame, (BIRD_WIDTH, BIRD_HEIGHT))

        # ----- (1) 주행용 버드뷰 / 차선 인식 -----
        bird = cv2.warpPerspective(frame_small, M, (BIRD_WIDTH, BIRD_HEIGHT))
        center_x, is_horizontal_bar, dbg = detect_lane_center(bird)
        view = bird.copy()

        if center_x is not None:
            error = center_x - FRAME_CENTER_X

            steering = pid.compute(error)
            steering *= 0.5
            steering = float(np.clip(steering, -MAX_STEERING, MAX_STEERING))
            if is_horizontal_bar:
                steering = 0

            L = int(np.clip(BASE_SPEED + steering, 0, 127))
            Rm = int(np.clip(BASE_SPEED - steering, 0, 127))

            tiki.set_motor_power(tiki.MOTOR_LEFT, L)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, Rm)
        else:
            tiki.stop()

        panel = make_panel(view, dbg)

        # ----- (2) 객체 인식 (YOLO + ArUco) : N프레임마다만 수행 -----
        frame_for_aruco = frame.copy()

        # (A) ArUco 포인트 감지 (원본 해상도 사용)
        corners, ids, _ = aruco_detector.detectMarkers(frame_for_aruco)

        if ids is not None:
            ids_list = [int(i[0]) for i in ids]

            for marker_id in ids_list:
                if marker_id in marker_to_point:
                    info = marker_to_point[marker_id]
                    # only treat 'point' kinds as mission points (Alpha/Bravo/Charlie/Finish)
                    if info['kind'] == 'point':
                        point_name = info['name']
                        current_point = point_name  # 현재 point 업데이트

                        if point_name not in visited_points:
                            visited_points.add(point_name)
                            transmitted_points.append(point_name)
                            
                            # 해당 point까지의 누적 detection 정보를 detection 형식으로 변환
                            # sendjson.ipynb 형식: 각 point별로 배열이 있어야 함 (빈 배열이라도)
                            detection_for_payload = {}
                            
                            # 모든 point에 대해 누적 detection 정보 추가
                            for pt in transmitted_points:
                                detection_list = []
                                if pt in point_detections:
                                    # 해당 point의 누적 detection 정보를 배열 형식으로 변환
                                    for class_name, count in point_detections[pt].items():
                                        if count > 0:  # count가 0보다 큰 경우만 추가
                                            detection_list.append({"type": class_name, "count": count})
                                detection_for_payload[pt] = detection_list
                            
                            print(f"[POINT] {point_name} 최초 통과 → 대시보드로 전송 (points 업데이트, detection: {detection_for_payload})")
                            # send the accumulated points list and detection in payload
                            send_to_server(points=transmitted_points, detected_objects=detection_for_payload)
                    else:
                        # sector handling: fire_buildings에 포함된 sector면 이미지 캡처 및 전송
                        sector_name = info['name']
                        print(f"[SECTOR] 감지: {sector_name}")
                        
                        # JSON 파일에서 fire_buildings 확인
                        try:
                            with open(JSON_PATH, 'r', encoding='utf-8') as f:
                                json_data = json.load(f)
                            
                            fire_buildings = json_data.get("fire_buildings", [])
                            mission_code = json_data.get("mission_code", "A3R8")
                            
                            # fire_buildings에 포함되어 있고, 아직 전송하지 않은 sector인 경우
                            if sector_name in fire_buildings and sector_name not in visited_sectors:
                                visited_sectors.add(sector_name)
                                
                                # sector 이름에서 숫자 추출 (예: "sector8" -> "8")
                                sector_number = re.search(r'\d+', sector_name)
                                if sector_number:
                                    sector_num = sector_number.group()
                                    
                                    # 이미지 파일 이름 생성: A3R8_sector8.jpg
                                    image_filename = f"{mission_code}_sector{sector_num}.jpg"
                                    image_path = os.path.join("/tmp", image_filename)
                                    
                                    # 현재 프레임 캡처
                                    cv2.imwrite(image_path, frame_for_aruco)
                                    print(f"[FIRE BUILDING] {sector_name} 감지 → 이미지 캡처: {image_filename}")
                                    
                                    # 서버로 이미지 전송
                                    send_dashboard_image(image_path)
                                    
                                    # 임시 파일 삭제 (선택사항)
                                    try:
                                        os.remove(image_path)
                                    except:
                                        pass
                                else:
                                    print(f"[WARNING] sector 이름에서 숫자를 추출할 수 없습니다: {sector_name}")
                        except FileNotFoundError:
                            print(f"[WARNING] JSON 파일을 찾을 수 없습니다: {JSON_PATH}")
                        except Exception as e:
                            print(f"[ERROR] fire_building 확인 중 오류: {e}")

            cv2.aruco.drawDetectedMarkers(frame_for_aruco, corners, ids)

        # (B) YOLO 객체 감지
        if frame_idx % 5 == 0:
            frame_for_yolo = frame.copy()
            results = model(
                frame_for_yolo,
                conf=0.45,
            )

            detected_counts = defaultdict(int)
            if results and len(results[0].boxes) > 0:
                classes = results[0].boxes.cls.cpu().numpy()
                
                # 객체가 인식됐을 때 current_point가 None이면 첫번째 point인 Alpha로 설정
                if current_point is None:
                    current_point = "Alpha"
                    print(f"[DETECTION] current_point가 None이었으므로 Alpha로 설정")
                
                for cls in classes:
                    class_name = object_names[int(cls)]
                    detected_counts[class_name] += 1
                    
                    # 현재 point에 detection 정보 누적
                    point_detections[current_point][class_name] += 1
                        
            if len(detected_counts) > 0:
                print(f"[DETECTION] 객체 감지: {dict(detected_counts)} (현재 point: {current_point})")

            annotated = results[0].plot()
            cv2.putText(
                annotated,
                "Object Detection",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, 0),
                2,
                cv2.LINE_AA,
            )

        # ----- 포트홀 감지 (적응형) -----
        check_and_handle_pothole(bird, tiki)

        # ----- (3) 두 화면을 Jupyter 위젯으로 출력 -----
        drive_widget.value = frame_to_bytes_drive(panel)
        if annotated is not None and frame_idx % 2 == 0:
            yolo_widget.value = frame_to_bytes_yolo(annotated)

        time.sleep(0.02)

except KeyboardInterrupt:
    tiki.stop()
    cap.release()
    cv2.destroyAllWindows()
    print("[END] 프로그램 종료")


로봇 초기화 완료 (모터 정지)
GST_ARGUS: Cleaning up
CONSUMER: Done Success
GST_ARGUS: Done Success
GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3264 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3264 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 12

[ WARN:0@298.703] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
[ WARN:0@298.708] global cap_gstreamer.cpp:2152 setProperty OpenCV | GStreamer warning: GStreamer: unhandled property


[START] 주행 + YOLO + ArUco 시스템 실행
[POTHOLE] 감지: white_frac=0.000 baseline=0.000 → 회피 수행
[AVOID] 포트홀 회피 시작 (좌회전)
[AVOID] 회피 완료

0: 320x416 (no detections), 72.0ms
Speed: 6.9ms preprocess, 72.0ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 74.7ms
Speed: 6.9ms preprocess, 74.7ms inference, 3.4ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 77.6ms
Speed: 7.3ms preprocess, 77.6ms inference, 2.8ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 74.8ms
Speed: 7.0ms preprocess, 74.8ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 74.9ms
Speed: 6.7ms preprocess, 74.9ms inference, 3.1ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 75.1ms
Speed: 6.9ms preprocess, 75.1ms inference, 2.7ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 74.9ms
Speed: 7.0ms preprocess, 74.9ms inference